In [19]:
import numpy as np

In [20]:
def traditional_multiply_matrix(A, B):
    result = np.zeros((A.shape[0], B.shape[1]))

    k = A.shape[1]
    
    for i in range(A.shape[0]):
        for j in range(B.shape[1]):
            sum_j = 0
            for k in range(A.shape[1]):
                sum_j += A[i, k] * B[k, j]
            
            result[i, j] = sum_j

    return result

In [39]:
def multiply_matrix(A, B, l):
    try:
        if A.shape[1] != B.shape[0]:
            raise ValueError("Liczba kolumn pierwszej macierzy musi być równa liczbie wierszy drugiej macierzy")
    
        if A.shape[0] < 2**l and A.shape[1] < 2**l and B.shape[0] < 2**l and B.shape[1] < 2**l:
            return traditional_multiply_matrix(A, B)
    except ValueError as e:
        print(f"Błąd: {e}")

## Tests

In [22]:
A = np.array([[1, 3], [2, 2]])
A

array([[1, 3],
       [2, 2]])

In [23]:
B = np.array([[2, 2], [6, 3]])
B

array([[2, 2],
       [6, 3]])

In [24]:
traditional_multiply_matrix(A, B)

array([[20., 11.],
       [16., 10.]])

In [25]:
multiply_matrix(A, B, 2)

array([[20., 11.],
       [16., 10.]])

In [34]:
C = np.array([[1, 2]])
C

array([[1, 2]])

In [40]:
multiply_matrix(A, C, 2)

Błąd: Liczba kolumn pierwszej macierzy musi być równa liczbie wierszy drugiej macierzy
